In [1]:
import os
import sys
import time

from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.evaluation import SimilarityFunction

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert-test", logfilename="../../log/s-bert-test")
device = GPU_info()
seed=111
seed_everything(seed)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


logfilepath:../../log/s-bert-test_2022-10-21.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
## 평가 설정 parameter 들 ########################################

# 평가할 s-bert 모델 경로
#smodel_path = "bert-base-multilingual-cased"
smodel_path = "../../data11/model/distilbert/bert-re-kowiki-mecab"

# 유사도 측정방식(COSINE, EUCLIDEAN, MANHATTAN, DOT_PRODUCT 중 선택 , 모두 spearman 방식임)
# => None 이면 아래 값들중 MAX 값 추력함
#main_similarity = None
main_similarity = SimilarityFunction.COSINE
#main_similarity = SimilarityFunction.EUCLIDEAN
#main_similarity = SimilarityFunction.MANHATTAN
#main_similarity = SimilarityFunction.DOT_PRODUCT

# 평가시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
output_path = 'eval'
os.makedirs(output_path, exist_ok=True)
csvfilename = 'V3.1.1'   # 유사도 저장할 파일명 (eval/similarity_evaluation_{csffilename}.csv 형식으로 저장)

# 평가 sts 형태의 말뭉치 파일들
use_korsts = 1 #  korsts 파일
use_kluests = 0  # kluests_v1.1 파일
use_glue_sts = 0 # true이면 영문 glue_sts 데이터셋 추가하여 평기 시킴
use_en_sts = 0    # true이면 영문 stsb_multi_mt데이터셋 추가하여 평가시킴.
   
# korsts tsv 파일 경로
korsts_file = '../../data11/korpora/korsts/tune_test.tsv'

# kluests json 파일 경로
kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

eval_batch_size = 64 # 배치파일사이즈 = 크면 eval 시간이 단축됨.단 gpu 메모리 올라감.

In [3]:
from datasets import load_dataset
test_samples = []


# /korsts/tune_test.tsv 파일을 불러옴
if use_korsts == True:
    with open(korsts_file, 'rt', encoding='utf-8') as fIn1:
        lines = fIn1.readlines()
        for line in lines:
            s1, s2, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0
            test_samples.append(InputExample(texts=[s1,s2], label=score))


# /klue-sts/klue-sts-v1.1_dev.json 파일을 불러옴
if use_kluests == True:
    import json

    with open(kluests_file, "r", encoding='utf-8') as fIn2:
        data = json.load(fIn2)
        for el in data:
            s1 = el["sentence1"]
            s2 = el["sentence2"]
            score = el["labels"]['label']
            test_samples.append(InputExample(texts=[s1,s2], label=score))
        
# 영문 stsb_multi_mt test 버전 파일 로딩함
if use_en_sts == True:
    en_sts_dataset = load_dataset("stsb_multi_mt", name="en", split="test")
    for data in en_sts_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["similarity_score"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
        
if use_glue_sts == True:   
    #glue stsb valildation 데이터셋 불러옴(subset : "stsb" = 1,500)
    glue_stsb_dataset = load_dataset("glue","stsb", split="validation")
    for data in glue_stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
    
print(test_samples[0:3])
print(f'*test_samples_len: {len(test_samples)}')

[<sentence_transformers.readers.InputExample.InputExample object at 0x7f80901e2e20>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f80901e2fa0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f808e4550d0>]
*test_samples_len: 1379


In [4]:
##############################################################################
# sentence bert 를 불러옴
# => 훈련되어서 저장된 s-bert 모델을 불러와서 성능 평가 해봄
# => 평가 내용은 해당 모델 경로에 'similarity_evaluation_korstr_tune_test_results.csv' 파일에 기록됨
# => 모델 다운로드 폴더 지정 : cache_folder=경로 
##############################################################################

logger.info("\n")
logger.info("======================TEST===================")

model = SentenceTransformer(smodel_path)
#model = SentenceTransformer(smodel_path, device='cpu')
#model = SentenceTransformer(smodel_path, cache_folder=output_path)
#model.to(device)
logger.info(f'{model}')

2022-10-21 13:08:25,561 - s-bert-test - INFO - 

2022-10-21 13:08:25,564 - s-bert-test - INFO - ======================TEST===================


TypeError: __init__() got an unexpected keyword argument 'max_seq_length'

In [ ]:
logger.info(f"*main_similarity: {main_similarity}")
logger.info(f"*model path: {smodel_path}")

if use_korsts == True:
    logger.info(f"*korsts file({korsts_file})")

if use_kluests == True:
    logger.info(f"*klue file({kluests_file})")
    
if use_en_sts == True:
    logger.info(f"*stsb_multi_mt")
  
if use_glue_sts == True:
    logger.info(f"*glue(stsb)")
    
start = time.time()

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, main_similarity=main_similarity, batch_size=eval_batch_size, name=csvfilename, show_progress_bar=True)
result = test_evaluator(model, output_path=output_path)
logger.info(f"\n")
        
logger.info(f'=== result: {result} ===')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info("=====================================================")
logger.info("\n")
